In [3]:
import numpy as np
import pandas as pd
import json
import glob
import gzip

# jsonl files

In [4]:
filename = "raw.jsonl"
with open(filename, 'r') as file:
    data = [json.loads(line) for line in file]
df = pd.DataFrame(data)
display(df.head())

,combat_id,event_type,timestamp,message_id,author_id,author_name,author_bot,created_at,content,embeds,...,targets,interaction_id,automation,automation_result,interaction_message_id,button_id,button_label,snippet_name,snippet_body,content_after
0,1669412158-d5e74f94-147e-4735-849b-7174b8aa8be0,message,1.669412e+09,1.045815e+18,17485940095965079179,Player 0,False,1.669412e+09,> *As the six lean over the murky pool and pee...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1669412158-d5e74f94-147e-4735-849b-7174b8aa8be0,message,1.669412e+09,1.045815e+18,7583945656211615731,Player 1,False,1.669412e+09,"""Too murky to make anything proper out of it. ...",[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1669412158-d5e74f94-147e-4735-849b-7174b8aa8be0,message,1.669412e+09,1.045815e+18,7921769374058877842,Avrae,True,1.669412e+09,```Awaiting combatants...```,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1669412158-d5e74f94-147e-4735-849b-7174b8aa8be0,message,1.669412e+09,1.045815e+18,7921769374058877842,Avrae,True,1.669412e+09,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1669412158-d5e74f94-147e-4735-849b-7174b8aa8be0,combat_start,1.669412e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(len(df))
print(len(df["human_readable"].dropna()))

342
57


In [6]:
df_temp = df["human_readable"].dropna()
df_temp = df_temp.astype('string')
df_temp = df_temp.str.replace(r"```md\nCurrent initiative: \d+ \(round \d+\)\n=====+\n", "", regex=True)
df_temp = df_temp.str.replace(r"```", "", regex=True)
df_temp = df_temp.str.replace(r"\d+: ", "", regex=True)
df_temp = df_temp.str.replace(r"# ", "", regex=True)
print(df_temp.dtype)
for value in df_temp:
    print(value)
    print("#################")

string

#################
  Keya <24/24 HP> (AC 18)
#################
  Bartholomew <23/23 HP> (AC 12)
  Keya <24/24 HP> (AC 18)
#################
  Bartholomew <23/23 HP> (AC 12)
  Aleksandra <15/15 HP> (AC 15)
  Keya <24/24 HP> (AC 18)
#################
  Verity Silverdust <18/18 HP> (AC 13)
  Bartholomew <23/23 HP> (AC 12)
  Aleksandra <15/15 HP> (AC 15)
  Keya <24/24 HP> (AC 18)
#################
  Verity Silverdust <18/18 HP> (AC 13)
  Nitar <31/35 HP> (AC 15)
  Bartholomew <23/23 HP> (AC 12)
  Aleksandra <15/15 HP> (AC 15)
  Keya <24/24 HP> (AC 18)
#################
  Verity Silverdust <18/18 HP> (AC 13)
  Nitar <31/35 HP> (AC 15)
  Bartholomew <23/23 HP> (AC 12)
  Aleksandra <15/15 HP> (AC 15)
  Keya <24/24 HP> (AC 18)
   SH1 <52/52 HP> 
#################
  Verity Silverdust <18/18 HP> (AC 16, Mage Armor [8.0 hours])
  Nitar <31/35 HP> (AC 15)
  Bartholomew <23/23 HP> (AC 12)
  Aleksandra <15/15 HP> (AC 15)
  Keya <24/24 HP> (AC 18)
   SH1 <52/52 HP> 
#################
  Verity 

In [7]:
# Regular expression to extract data
pattern = r"^(?P<Name>.+?)\s*<(?P<Current_HP>-?\d+)/(?P<Max_HP>\d+)\s*HP>\s*(\(AC\s*(?P<AC>\d+)[^)]*\))?\s*(?P<Effects>.*)?"
df_extracted = df_temp.str.extract(pattern)
df_extracted

,Name,Current_HP,Max_HP,3,AC,Effects
5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,Keya,24,24,(AC 18),18,
15,Bartholomew,23,23,(AC 12),12,Keya <24/24 HP> (AC 18)
16,Bartholomew,23,23,(AC 12),12,Aleksandra <15/15 HP> (AC 15)
18,Verity Silverdust,18,18,(AC 13),13,Bartholomew <23/23 HP> (AC 12)
25,Verity Silverdust,18,18,(AC 13),13,Nitar <31/35 HP> (AC 15)
29,Verity Silverdust,18,18,(AC 13),13,Nitar <31/35 HP> (AC 15)
34,Verity Silverdust,18,18,"(AC 16, Mage Armor [8.0 hours])",16,Nitar <31/35 HP> (AC 15)
38,Verity Silverdust,18,18,"(AC 16, Mage Armor [8.0 hours])",16,Nitar <31/35 HP> (AC 15)
44,Verity Silverdust,18,18,"(AC 16, Mage Armor [8.0 hours])",16,Nitar <31/35 HP> (AC 15)


# jsonl.gz files

In [8]:
file_pattern = './anonymized/data/*.jsonl.gz'
files = glob.glob(file_pattern)
print(len(files))
all_data = []

for filename in files[:10]:
    with gzip.open(filename, 'rt', encoding='utf-8') as file:
        all_data.extend(json.loads(line) for line in file)

df = pd.DataFrame(all_data)
df

19


,combat_id,event_type,timestamp,message_id,author_id,author_name,author_bot,created_at,content,embeds,...,content_before,content_after,interaction_id,automation,automation_result,interaction_message_id,button_id,button_label,snippet_name,snippet_body
0,1669403645-7aaa8604-e567-4004-9e3b-2fb153d2c92d,message,1.669404e+09,1.045779e+18,912920741610430984,Player 0,True,1.669404e+09,**as Erien asks the apparent leader of the orc...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1669403645-7aaa8604-e567-4004-9e3b-2fb153d2c92d,message,1.669404e+09,1.045779e+18,261302296103747584,Avrae,True,1.669404e+09,```Awaiting combatants...```,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1669403645-7aaa8604-e567-4004-9e3b-2fb153d2c92d,message,1.669404e+09,1.045779e+18,261302296103747584,Avrae,True,1.669404e+09,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1669403645-7aaa8604-e567-4004-9e3b-2fb153d2c92d,combat_start,1.669404e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1669403645-7aaa8604-e567-4004-9e3b-2fb153d2c92d,combat_state_update,1.669404e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4287,1669400411-a21252c1-761a-4594-b463-9892bedd8298,message,1.669401e+09,1.045770e+18,261302296103747584,Avrae,True,1.669401e+09,Are you sure you want to end combat? (Reply wi...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4288,1669400411-a21252c1-761a-4594-b463-9892bedd8298,message,1.669401e+09,1.045770e+18,272697093190263629,Player 1,False,1.669401e+09,yes,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4289,1669400411-a21252c1-761a-4594-b463-9892bedd8298,message,1.669401e+09,1.045770e+18,261302296103747584,Avrae,True,1.669401e+09,"OK, ending...",[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4290,1669400411-a21252c1-761a-4594-b463-9892bedd8298,combat_end,1.669401e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Filtered triples

In [9]:
filename = "filtered_triples.jsonl"
with open(filename, 'r') as file:
    data = [json.loads(line) for line in file]
    #print(data)
df = pd.DataFrame(data)
display(df.head())
display(df.describe())
display(df.describe(include='O'))

,speaker_id,before_utterances,combat_state_before,current_actor,commands_norm,automation_results,caster_after,targets_after,combat_state_after,after_utterances,utterance_history,before_idxs,before_state_idx,command_idxs,after_state_idx,after_idxs,embed_idxs
0,13086297142809228709,[> *As the six lean over the murky pool and pe...,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...",None,"[!cast ""mage armor""]",[Verity Silverdust casts Mage Armor!\nVerity S...,"{'name': 'Verity Silverdust', 'hp': '<18/18 HP...",[],"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...",[],[Player 0: > *As the six lean over the murky p...,[0],29,"[32, 33, 34, 36]",34,[],[35]
1,17485940095965079179,[> *The party is SURPRISED! SH1 will act first...,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', '...",[!i a appear -t moz],[SH1 has a Horrific Appearance!\nMozzie Urahak...,"{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', '...","[{'name': 'Mozzie Urahaka', 'hp': '<22/22 HP; ...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H...",[],[Player 0: > *As the six lean over the murky p...,[41],44,"[46, 47, 48, 50]",48,[],[49]
2,17485940095965079179,"[> *Cackling, it reaches out to place its othe...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', '...",[!i a appear -t verity -t nitar -t barth -t al...,[SH1 has a Horrific Appearance!\nVerity Silver...,"{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', '...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H...",[],[Player 0: > *As the six lean over the murky p...,"[57, 58]",48,"[59, 60, 61, 63]",61,[],[62]
3,13086297142809228709,"[""What did I say earlier? Moderate chance of v...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Verity Silverdust', 'hp': '<18/18 HP...",[!a rapier -t SH1],[Verity Silverdust attacks with a Rapier!\nVer...,"{'name': 'Verity Silverdust', 'hp': '<18/18 HP...","[{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', ...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H...",[],[Player 0: > *As the six lean over the murky p...,[68],66,"[69, 70, 71, 73]",71,[],[72]
4,13676033686500382670,[_Barty feels the chaos flow through them as t...,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Verity Silverdust', 'hp': '<18/18 HP...",[!a tide],[Bartholomew uses Tides of Chaos!\n],"{'name': 'Bartholomew', 'hp': '<23/23 HP; Heal...",[],"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...",[_But it made no difference at all!_],[Player 0: > *The party is SURPRISED! SH1 will...,[74],71,"[75, 76, 77, 79]",77,[83],[78]


,before_state_idx,after_state_idx
count,25.000000,25.000000
mean,174.320000,180.640000
std,91.144903,91.200731
min,29.000000,34.000000
25%,106.000000,111.000000
50%,170.000000,178.000000
75%,255.000000,262.000000
max,321.000000,329.000000


,speaker_id,before_utterances,combat_state_before,current_actor,commands_norm,automation_results,caster_after,targets_after,combat_state_after,after_utterances,utterance_history,before_idxs,command_idxs,after_idxs,embed_idxs
count,25,25,25,24,25,25,25,25,25,25,25,25,25,25,25
unique,6,24,14,13,21,21,12,13,18,12,23,24,25,12,25
top,13676033686500382670,[],"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Verity Silverdust', 'hp': '<18/18 HP...",[!a tide],[Keya attacks with a Whip!\nKeya attacked SH1 ...,"{'name': 'Verity Silverdust', 'hp': '<18/18 HP...",[],"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...",[],"[Player 3: _But, the day goes on, and the reac...",[],"[32, 33, 34, 36]",[],[35]
freq,7,2,3,4,3,3,4,9,3,14,2,2,1,14,1


In [10]:
df.columns

Index(['speaker_id', 'before_utterances', 'combat_state_before',
       'current_actor', 'commands_norm', 'automation_results', 'caster_after',
       'targets_after', 'combat_state_after', 'after_utterances',
       'utterance_history', 'before_idxs', 'before_state_idx', 'command_idxs',
       'after_state_idx', 'after_idxs', 'embed_idxs'],
      dtype='object')

In [11]:
df = df[['combat_state_before', 'current_actor', 'caster_after', 'targets_after', 'combat_state_after']]
display(df)
for index, row in df.iterrows():
    print(row['combat_state_before'])
    print(row['combat_state_after'])
    print("#########")


,combat_state_before,current_actor,caster_after,targets_after,combat_state_after
0,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...",None,"{'name': 'Verity Silverdust', 'hp': '<18/18 HP...",[],"[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."
1,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', '...","{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', '...","[{'name': 'Mozzie Urahaka', 'hp': '<22/22 HP; ...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."
2,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', '...","{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', '...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."
3,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Verity Silverdust', 'hp': '<18/18 HP...","{'name': 'Verity Silverdust', 'hp': '<18/18 HP...","[{'name': 'SH1', 'hp': '<52/52 HP; Healthy>', ...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."
4,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Verity Silverdust', 'hp': '<18/18 HP...","{'name': 'Bartholomew', 'hp': '<23/23 HP; Heal...",[],"[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."
5,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Nitar', 'hp': '<31/35 HP; Injured>',...","{'name': 'Nitar', 'hp': '<31/35 HP; Injured>',...",[],"[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."
6,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Bartholomew', 'hp': '<23/23 HP; Heal...","{'name': 'Bartholomew', 'hp': '<23/23 HP; Heal...","[{'name': 'SH1', 'hp': '<45/52 HP; Injured>', ...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."
7,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Bartholomew', 'hp': '<23/23 HP; Heal...","{'name': 'Bartholomew', 'hp': '<23/23 HP; Heal...",[],"[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."
8,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Bartholomew', 'hp': '<23/23 HP; Heal...","{'name': 'Bartholomew', 'hp': '<23/23 HP; Heal...",[],"[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."
9,"[{'name': 'Verity Silverdust', 'hp': '<18/18 H...","{'name': 'Aleksandra', 'hp': '<15/15 HP; Healt...","{'name': 'Aleksandra', 'hp': '<15/15 HP; Healt...","[{'name': 'SH1', 'hp': '<42/52 HP; Injured>', ...","[{'name': 'Verity Silverdust', 'hp': '<18/18 H..."


[{'name': 'Verity Silverdust', 'hp': '<18/18 HP; Healthy>', 'class': 'Rogue 3', 'race': 'Lightfoot Halfling', 'attacks': 'Dagger, Rapier, Shortbow, Unarmed Strike', 'spells': 'Sleep, Disguise Self, Message, Mage Armor, Prestidigitation, Mage Hand', 'actions': 'Steady Aim, Sneak Attack, Cunning Action', 'effects': '', 'description': 'Halfling, Female, ?? years\nHeight: 3\'1"\nWeight: 42lbs\nHair: Blonde\nEyes: Blue\nSkin: Fair\n\n*too lazy to fill out the rest of the desc, remind me to do it later*', 'controller_id': '13086297142809228709'}, {'name': 'Nitar', 'hp': '<31/35 HP; Injured>', 'class': 'Barbarian 3', 'race': 'Shifter', 'attacks': 'Crystal Spike (Greataxe), Handaxe, Javelin, Unarmed Strike', 'spells': '', 'actions': 'Rage, Form of the Beast: Bite (Heal), Form of the Beast: Tail (reaction), Form of the Beast: Claws, Form of the Beast: Tail, Shift, Wildhunt Shifting, Form of the Beast: Bite', 'effects': '', 'description': 'Standing at a formidable 6\'1", Nitar is a sturdily buil